##### Configurações Iniciais

In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, ConfusionMatrixDisplay
from scipy.stats import pointbiserialr, ttest_ind

from Modules.cemaden_loader import CemadenLoader, CemadenMerger
from Modules.forecast_loader import ForecastLoader
from Modules.variables_info import variables_info

In [10]:
# Parâmetros gerais
lat_porto = -25.500511
lon_porto = -48.519331

cemaden_path = '../Database/Paranagua_CEMADEN'
forecast_path = '../Database/forecast_data'
timebox = ('202405', '202412')

variaveis = variables_info['variaveis']
legendas = variables_info['legendas']
legendas_dict = dict(zip(variaveis, legendas))

# Repositorios de Saída
base_output = '../Output/Analise_01'
os.makedirs('../Output/Analise_01', exist_ok=True)
for subfolder in ['Tables', 'Imagens/Serie_Temporal', 'Imagens/MatrizConfusao', 'Imagens/Heatmap', 'Imagens/Metricas']:
    os.makedirs(os.path.join(base_output, subfolder), exist_ok=True)


##### Carregamento dos Dados

In [11]:
forecast = ForecastLoader(forecast_path, lat_target=lat_porto, lon_target=lon_porto)
df_forecast = forecast.load_forecasts(timebox, drop_duplicates=True)

cemaden = CemadenLoader(cemaden_path)
df_obs = cemaden.load(timebox)

merger = CemadenMerger(obs_df=df_obs, forecast_df=df_forecast)
df_merged = merger.merge_data()

##### Análises

In [12]:
categorias = {
    'Tx': [v for v in variaveis if 'smooth_rate' in v and 'minimum' not in v and 'maximum' not in v],
    'Min': [v for v in variaveis if 'minimum' in v],
    'Max': [v for v in variaveis if 'maximum' in v],
    'Prob': [v for v in variaveis if 'precipitation_probability' in v],
}
categorias_legendas = {k: [legendas_dict[v] for v in vs] for k, vs in categorias.items()}

###### Serie Temporal para avaliação das Estações

In [13]:
with sns.axes_style('whitegrid'):
    for estacao in df_merged['codEstacao'].unique():
        df_est = df_merged[df_merged['codEstacao'] == estacao]

        fig, axs = plt.subplots(4, 1, figsize=(16, 12), sharex=True)

        for i, (cat, vars_cat) in enumerate(categorias.items()):
            if cat != 'Prob':
                axs[i].plot(df_est['time'], df_est['precipacao_obs'], label='Chuva Observada', color='black', linewidth=1)

            legends = [l.replace(') ', ')\n') for l in categorias_legendas[cat]]

            for var, leg in zip(vars_cat, legends):
                axs[i].plot(df_est['time'], df_est[var], label=leg, linewidth=1.7, alpha=0.5)

            axs[i].set_ylabel('Probabilidade de Chuva (%)' if cat == 'Prob' else 'Precipitação (mm/h)')
            axs[i].grid(True)

            handles, labels = axs[i].get_legend_handles_labels()
            axs[i].legend(handles, labels, loc='center left', bbox_to_anchor=(1.01, 0.5), fontsize='small')

        axs[-1].tick_params(axis='x', rotation=45)
        axs[-1].set_xlim(df_est['time'].min(), df_est['time'].max())

        plt.suptitle(f'Comparação por Categoria — Estação {estacao}', fontsize=14, x=.4, y=0.95)
        plt.tight_layout(rect=[0, 0, 0.85, 0.96])

        plt.savefig(f'{base_output}/Imagens/Serie_temporal/serie_temporal_{estacao}.png', bbox_inches='tight')
        plt.close()

###### Abordagem de Avaliação

In [14]:
resultados = []

for estacao in df_merged['codEstacao'].unique():

    # Devido ao grande nº de falhas nos dados observados da estação 411820403A, optei por remove-la das análises
    if estacao == '411820403A':
        continue

    df_est = df_merged[df_merged['codEstacao'] == estacao]
    nome_estacao = df_est['nomeEstacao'].iloc[0]

    for var in variaveis:
        try:
            y_true = (df_est['precipacao_obs'] > 0).astype(int)
            y_pred_bin = (df_est[var] > 0).astype(int)
            y_score = df_est[var]

            # Métricas principais para avaliar se o modelo consegue prever corretamente os eventos de interesse (Chuvas)
            precision = precision_score(y_true, y_pred_bin, zero_division=0)
            recall = recall_score(y_true, y_pred_bin, zero_division=0)
            f1 = f1_score(y_true, y_pred_bin, zero_division=0)
            auc = roc_auc_score(y_true, y_score)

            # Estatísticas adicionais para avaliar se os duas amostras são estatisticamente diferentes entre si.
            corr, _ = pointbiserialr(y_true, y_score)
            grupo1 = y_score[y_true == 1]
            grupo2 = y_score[y_true == 0]
            t_stat, p_val = ttest_ind(grupo1, grupo2, equal_var=False)

            resultados.append({
                'codEstacao': estacao,
                'nomeEstacao': nome_estacao,
                'variavel': var,
                'legenda': legendas_dict.get(var, var),
                'precision': precision,
                'recall': recall,
                'f1_score': f1,
                'roc_auc': auc,
                'correlacao_bisserial': corr,
                't_stat': t_stat,
                'p_val': p_val
            })

            # Matriz de confusão
            fig, ax = plt.subplots(figsize=(6.5, 4))
            disp = ConfusionMatrixDisplay.from_predictions(
                y_true, y_pred_bin, ax=ax,
                cmap='Blues', colorbar=False,
                display_labels=['Não Choveu', 'Choveu']
            )
            for text in disp.text_.ravel():
                try:
                    val = float(text.get_text())
                    text.set_text(f'{int(val)}' if val.is_integer() else f'{val:.1f}')
                except:
                    pass
            ax.set_title(f'Matriz de Confusão\n{legendas_dict.get(var, var)} — {nome_estacao}')
            plt.ylabel('Observação')
            plt.xlabel('Modelo')
            plt.tight_layout()
            plt.savefig(f'{base_output}/Imagens/MatrizConfusao/{estacao}_{var}.png')
            plt.close()

        except Exception as e:
            print(f'Erro na variável {var} para estação {estacao}: {e}')

result_df = pd.DataFrame(resultados)
result_df['ordem'] = result_df['variavel'].apply(lambda x: variaveis.index(x))
result_df = result_df.sort_values(['codEstacao', 'ordem'])
result_df.to_csv(f'{base_output}/Tables/metricas_modelo.csv', index=False)

for estacao in result_df['codEstacao'].unique():
    df_estacao = result_df[result_df['codEstacao'] == estacao]
    nome = df_estacao['nomeEstacao'].iloc[0]
    heatmap_data = df_estacao.set_index('legenda')[['precision', 'recall', 'f1_score', 'roc_auc']]
    heatmap_data.columns = ['Precision', 'Recall', 'F1 Score', 'ROC AUC']

    plt.figure(figsize=(10, 6))
    sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu', fmt='.2f')
    plt.ylabel(None)
    plt.title(f'Métricas de Previsão — {nome}')
    plt.tight_layout()
    plt.savefig(f'{base_output}/Imagens/Heatmap/heatmap_{estacao}.png')
    plt.close()

##### Avaliação Inicial

| Métrica    | Boa se...	                | Interpretação                                                         |
|------------|------------------------------|-----------------------------------------------------------------------|
| Precision  | Alta (> 0.7)	                | O modelo acerta quando prevê chuva                                    |
| Recall     | Alta (> 0.7)	                | O modelo consegue capturar quase todas as chuvas reais                |
| F1 Score   | Alta (> 0.7)	                | Bom equilíbrio entre precisão e recall                                |
| ROC AUC    | Alta (> 0.8)	                | O modelo distingue bem entre chuva e não-chuva                        |
| Corr. Bis. | Alta positivamente (> 0.3)	| Previsão contínua se alinha bem com ocorrência observada              |
| Valor-p    | Baixo (< 0.05)           	| A diferença nas previsões entre chuva e não-chuva é significativa     |


In [15]:
df = pd.read_csv(f'{base_output}/Tables/metricas_modelo.csv')

df_sorted = df.sort_values(['codEstacao', 'f1_score'], ascending=[True, False])

top3_por_estacao = df_sorted.groupby('codEstacao').head(3)

for estacao, grupo in top3_por_estacao.groupby('codEstacao'):
    nome = grupo['nomeEstacao'].iloc[0]
    print(f'\n📍Estação: {estacao} – {nome}')
    display(grupo[['variavel', 'legenda', 'precision', 'recall', 'f1_score', 'roc_auc', 'correlacao_bisserial', 'p_val']])


📍Estação: 411820401A – Vila do Povo


,variavel,legenda,precision,recall,f1_score,roc_auc,correlacao_bisserial,p_val
5,lwe_precipitation_smooth_rate_minimum_20km,Tx. de Precipitacao (mm/h) Mín. em um Raio de ...,0.333333,0.292208,0.311419,0.630696,0.255449,3.679172e-06
0,lwe_precipitation_smooth_rate,Tx. de Precipitacao (mm/h),0.210773,0.584416,0.309811,0.739259,0.282342,2.554930e-09
1,lwe_precipitation_smooth_rate_minimum_10km,Tx. de Precipitacao (mm/h) Mín. em um Raio de ...,0.258065,0.363636,0.301887,0.655877,0.244495,9.891126e-07



📍Estação: 411820402A – Ponta do Caju


,variavel,legenda,precision,recall,f1_score,roc_auc,correlacao_bisserial,p_val
18,lwe_precipitation_smooth_rate_minimum_20km,Tx. de Precipitacao (mm/h) Mín. em um Raio de ...,0.338583,0.341270,0.339921,0.656733,0.335301,0.000008
14,lwe_precipitation_smooth_rate_minimum_10km,Tx. de Precipitacao (mm/h) Mín. em um Raio de ...,0.263682,0.420635,0.324159,0.686648,0.334989,0.000001
20,lwe_precipitation_smooth_rate_minimum_35km,Tx. de Precipitacao (mm/h) Mín. em um Raio de ...,0.428571,0.238095,0.306122,0.611899,0.289778,0.000195


In [16]:
melhores_f1 = df.groupby('variavel')[['f1_score']].max().reset_index()
melhores_f1['legenda'] = melhores_f1['variavel'].map(dict(zip(df['variavel'], df['legenda'])))

plt.figure(figsize=(10, 6))
sns.barplot(
    data=melhores_f1.sort_values('f1_score', ascending=False),
    x='f1_score', y='legenda', hue='legenda',
    palette='Blues_d', legend=False
    )
plt.title('Top variáveis por F1 Score (melhor desempenho)')
plt.xlabel('F1 Score')
plt.ylabel(None)
plt.xlim(0, 1)
plt.tight_layout()
plt.savefig(f'{base_output}/Imagens/Metricas/f1_barplot.png')
plt.close()

# Heatmap
media_f1 = df.groupby('legenda')['f1_score'].mean().sort_values(ascending=False)
df['legenda'] = pd.Categorical(df['legenda'], categories=media_f1.index, ordered=True)

heatmap_data = df.pivot(index='legenda', columns='nomeEstacao', values='f1_score')
plt.figure(figsize=(10, len(heatmap_data) * 0.5))
sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu', fmt='.2f', linewidths=0.5, linecolor='white')
plt.title('F1 Score por Variável e Estação (Ordenado pela Média de F1)', fontsize=14)
plt.xlabel('Estação')
plt.ylabel(None)
plt.tight_layout()
plt.savefig(f'{base_output}/Imagens/Metricas/heatmap.png')
plt.close()

##### Resultados

- *As variáveis com taxa de precipitação mínima em raio de 10km ou 20km estão entre as melhores em ambas as estações.*

- *A taxa de precipitação direta (lwe_precipitation_smooth_rate) também performa bem, especialmente em Vila do Povo.*